In [1]:
import sys
sys.path.extend(['..', '../', '../..', '../../' ])
%load_ext autoreload
%autoreload 2

# ResNet50 - MNIST

In [2]:
import paths
import keras
from src.parameters import metrics
from src.datasets.dataset_generators import DatasetGenerators
from src.keras_callback.time_history import TimeHistory
from src.export.csv_export import export_learning_history_to_csv, export_learning_validation_result_to_csv
from src.export.model_export import clear_exported_models
from src.keras_model.resnet50_network import ResNet50Network

Using TensorFlow backend.


In [3]:
network_name = 'resnet50'
fit_epochs = 50
dataset = 'mnist'
dataset_percentage = '5'

In [4]:
network = ResNet50Network()

In [5]:
dataset_generators = DatasetGenerators(
    name=f'{dataset}_{dataset_percentage}',
    dataset_dir=paths.data_location,
    image_size=network.image_size,
    batch_size=network.batch_size,
    grayscale=network.grayscale
)

Loading train set for mnist_5...
Found 2419 images belonging to 10 classes.
Loading test set for mnist_5...
Found 695 images belonging to 10 classes.
Loading validation set for mnist_5...
Found 349 images belonging to 10 classes.


In [6]:
model = network.get_compiled_model(
    classes=dataset_generators.test.num_classes
)

In [7]:
checkpoint_path = f'{paths.models_location}{network_name}/{dataset}/'
best_model_path = f'{checkpoint_path}best.h5'
clear_exported_models(checkpoint_path)

checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=f'{checkpoint_path}{{epoch:02d}}-{{val_accuracy}}.h5',
    save_best_only=False,
    save_weights_only=False
)

best_callback = keras.callbacks.ModelCheckpoint(
    filepath=best_model_path,
    save_best_only=True,
    verbose=1,
    save_weights_only=False
)

time_callback = TimeHistory()

model_callbacks = [checkpoint_callback, best_callback, time_callback]

In [ ]:
fit_result = model.fit_generator(
    generator=dataset_generators.train,
    epochs=fit_epochs,
    validation_data=dataset_generators.test,
    callbacks = model_callbacks
)

Epoch 1/50
75/76 [============================>.] - ETA: 7s - loss: 0.7985 - accuracy: 0.7491 - precision: 0.8608 - recall: 0.8727 

In [ ]:
best_model = keras.models.load_model(best_model_path,
    custom_objects= {
        'precision': metrics.precision,
        'recall': metrics.recall
    }
)
validation_result = best_model.evaluate_generator(
    generator=dataset_generators.validate
)

## MNIST Results

### Epoch CSV

In [ ]:
csv_file_path = f'{paths.results_location}{network_name}/{dataset}-history.csv'
export_learning_history_to_csv(csv_file_path, fit_result, time_callback.times)

### Validation result

In [ ]:
csv_file_path = f'{paths.results_location}{network_name}/{dataset}-result.csv'
export_learning_validation_result_to_csv(csv_file_path, validation_result, time_callback.times)